In [2]:
import pandas as pd

# Left DataFrame: Student details
df1 = pd.DataFrame({
    "StudentID": [1, 2, 3],
    "Name": ["Ravi", "Anjali", "Kiran"]
})

# Right DataFrame: Science Marks
df2 = pd.DataFrame({
    "StudentID": [1, 2, 4],
    "Science": [80, 75, 90]
})

print("DF1 (Student Details):\n", df1)
print("\nDF2 (Science Marks):\n", df2)


DF1 (Student Details):
    StudentID    Name
0          1    Ravi
1          2  Anjali
2          3   Kiran

DF2 (Science Marks):
    StudentID  Science
0          1       80
1          2       75
2          4       90


In [4]:
#Inner Join
inner= pd.merge(df1,df2,on="StudentID", how="inner")

In [5]:
inner

,StudentID,Name,Science
0,1,Ravi,80
1,2,Anjali,75


In [6]:
left= pd.merge(df1,df2, on="StudentID", how="left")

In [7]:
left

,StudentID,Name,Science
0,1,Ravi,80.0
1,2,Anjali,75.0
2,3,Kiran,NaN


In [8]:
right=pd.merge(df1,df2,on="StudentID",how="right")

In [9]:
right

,StudentID,Name,Science
0,1,Ravi,80
1,2,Anjali,75
2,4,NaN,90


In [10]:
outer= pd.merge(df1,df2,on="StudentID",how="outer")


In [11]:
outer

,StudentID,Name,Science
0,1,Ravi,80.0
1,2,Anjali,75.0
2,3,Kiran,NaN
3,4,NaN,90.0


In [12]:
#Merging — quick advanced moves (theory + hands-on)


In [13]:
#1) Different key names
dfA = pd.DataFrame({"ID":[1,2,3], "Name":["Ravi","Anjali","Kiran"]})
dfB = pd.DataFrame({"student_id":[1,2,4], "Maths":[81,76,90]})

merged = pd.merge(dfA, dfB, left_on="ID", right_on="student_id", how="left")
merged = merged.drop(columns="student_id")


In [14]:
merged

,ID,Name,Maths
0,1,Ravi,81.0
1,2,Anjali,76.0
2,3,Kiran,NaN


In [15]:
#2) Overlapping column names → use suffixes
dfC = pd.DataFrame({"StudentID":[1,2,3], "City":["Pune","Delhi","Goa"]})
dfD = pd.DataFrame({"StudentID":[2,3,4], "City":["Noida","Panaji","Hyderabad"]})

city_merge = pd.merge(dfC, dfD, on="StudentID", how="outer", suffixes=("_old","_new"))


In [16]:
city_merge

,StudentID,City_old,City_new
0,1,Pune,NaN
1,2,Delhi,Noida
2,3,Goa,Panaji
3,4,NaN,Hyderabad


In [17]:
#3) See which rows matched (debugging!) → indicator=True
dbg = pd.merge(dfA, dfB, left_on="ID", right_on="student_id",
               how="outer", indicator=True)
# Rows that didn’t match on both sides
unmatched = dbg[dbg["_merge"] != "both"]


In [18]:
unmatched

,ID,Name,student_id,Maths,_merge
2,3.0,Kiran,NaN,NaN,left_only
3,NaN,NaN,4.0,90.0,right_only


In [19]:
dbg

,ID,Name,student_id,Maths,_merge
0,1.0,Ravi,1.0,81.0,both
1,2.0,Anjali,2.0,76.0,both
2,3.0,Kiran,NaN,NaN,left_only
3,NaN,NaN,4.0,90.0,right_only


In [20]:
#) Guard against accidental cartesian explosions → validate=
# Expect strict one-to-one key match:
clean = pd.merge(dfA, dfB, left_on="ID", right_on="student_id",
                 how="inner", validate="one_to_one")
# Valid options: "one_to_one", "one_to_many", "many_to_one", "many_to_many"


In [21]:
clean

,ID,Name,student_id,Maths
0,1,Ravi,1,81
1,2,Anjali,2,76


In [22]:
#5) Concatenation (stacking same-schema tables)
term1 = pd.DataFrame({"StudentID":[1,2], "Science":[80,75]})
term2 = pd.DataFrame({"StudentID":[3,4], "Science":[68,90]})

stacked = pd.concat([term1, term2], ignore_index=True)  # row-wise


In [23]:
stacked

,StudentID,Science
0,1,80
1,2,75
2,3,68
3,4,90


Mini Practicals

In [24]:
#p1) Merge dfA & dfB (different key names) with a left join, keep only ["ID","Name","Maths"].

In [38]:
p1merge=pd.merge(dfA,dfB, left_on="ID", right_on="student_id", how ="left")
p1merge=p1merge.drop(columns="student_id")

In [39]:
p1merge

,ID,Name,Maths
0,1,Ravi,81.0
1,2,Anjali,76.0
2,3,Kiran,NaN


In [43]:
#P2) Using city_merge above, show only rows where the city changed 
#(i.e., City_old != City_new and neither is NaN).

In [50]:
filter_city=city_merge[city_merge["City_old"]!=city_merge["City_new"]].dropna()
filter_city

,StudentID,City_old,City_new
1,2,Delhi,Noida
2,3,Goa,Panaji


In [52]:
#P3. Do an outer merge of dfA & dfB 
#with indicator=True, and print only the rows coming from right_only.

In [61]:
p3m= pd.merge(dfA, dfB, left_on="ID", right_on="student_id", how="outer", indicator=True)
p3m =p3m[p3m["_merge"]=="right_only"]
p3m

,ID,Name,student_id,Maths,_merge
3,NaN,NaN,4.0,90.0,right_only


In [62]:
#P4. Create dfDup = pd.DataFrame({"ID":[1,1,2], "Hobby":["Chess","Cricket","Music"]})
#Merge dfA with dfDup on ID using validate="one_to_one" and observe what happens; 
#then fix it with the correct validate mode.

In [63]:
dfDup = pd.DataFrame({"ID":[1,1,2], "Hobby":["Chess","Cricket","Music"]})

In [72]:
dp4= pd.merge(dfA, dfDup, on="ID", how="inner", validate="one_to_many")
dp4

,ID,Name,Hobby
0,1,Ravi,Chess
1,1,Ravi,Cricket
2,2,Anjali,Music


In [73]:
dfe = pd.DataFrame({"ID":[1,1,2], "Hobby":["Chess","Cricket","Music"]})
dff = pd.DataFrame({"ID":[1,1,2], "Hobby":["Chess","Cricket","Music"]})

In [75]:
dp5= pd.merge(dfe, dff, on="ID", how="inner")
dp5

,ID,Hobby_x,Hobby_y
0,1,Chess,Chess
1,1,Chess,Cricket
2,1,Cricket,Chess
3,1,Cricket,Cricket
4,2,Music,Music
